<table class="ee-notebook-buttons" align="left">
    <td><a target="_blank"  href="https://github.com/giswqs/geemap/tree/master/examples/template/template.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td>
    <td><a target="_blank"  href="https://nbviewer.jupyter.org/github/giswqs/geemap/blob/master/examples/template/template.ipynb"><img width=26px src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/38/Jupyter_logo.svg/883px-Jupyter_logo.svg.png" />Notebook Viewer</a></td>
    <td><a target="_blank"  href="https://colab.research.google.com/github/giswqs/geemap/blob/master/examples/template/template.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a></td>
</table>

# Clasificación supervisada SVM
Script trabajado por Francisco Albornoz (2021-04-13) / Última modificación / Análisis y clasificación supervisada región de Aysen

<img src=https://www.ecopaysen.cl/wp-content/uploads/2019/12/ecopaysen_marca_web.png>

In [ ]:
# Installs geemap package
# import subprocess

# try:
#     import geemap
# except ImportError:
#     print("Installing geemap ...")
#     subprocess.check_call(["python", "-m", "pip", "install", "geemap"])

In [1]:
import os
import ee
import geemap
import xarray_leaflet
import rasterio 
from rasterio import plot
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

## Crear un mapa interactivo 
The default basemap is `Google Maps`. [Additional basemaps](https://github.com/giswqs/geemap/blob/master/geemap/basemaps.py) can be added using the `Map.add_basemap()` function. 

In [2]:
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

## Escogemos la imagen landsat 

In [3]:
centroid = ee.Geometry.Point([-72, -44])

landsat = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(centroid) \
    .filterDate('2020-01-01', '2021-03-31') \
    .sort('CLOUD_COVER') \
    .first() \
    .select('B[1-7]')

landsat_vis = {
    'min': 0,
    'max': 3000,
    'bands': ['B5', 'B4', 'B3']
}

Map.centerObject(centroid, 8)
Map.addLayer(landsat, landsat_vis, "Landsat-8")


## Importar rasters locales 

In [ ]:
%pwd

In [ ]:
# verificar en el sistema del pc cual es la ruta
# os.path.expanduser('/mnt/c/Users/R7 3700X/Documents/GitHub/VitoAlbornoz/') 

In [4]:
# verificar en el sistema del pc cual es la ruta
out_dir = os.path.join(os.path.expanduser('/mnt/c/Users/R7 3700X/Documents/GitHub/VitoAlbornoz/'), 'Data')
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

raster_aysen = os.path.join(out_dir, 'landcover_2018_aysen.tif')

In [5]:
out_dir

'/mnt/c/Users/R7 3700X/Documents/GitHub/VitoAlbornoz/Data'

In [6]:
raster_aysen

'/mnt/c/Users/R7 3700X/Documents/GitHub/VitoAlbornoz/Data/landcover_2018_aysen.tif'

In [7]:
Map.add_raster(raster_aysen, colormap= 'terrain', layer_name='Landcover 2018')
Map

Map(bottom=42006.0, center=[-44, -72], controls=(WidgetControl(options=['position'], widget=HBox(children=(Tog…

In [ ]:
landcover = ee.Image("users/franciscoalbornoz/landcover_sur")
landcover

In [8]:
raster = rasterio.open(out_dir+'/landcover_2018_aysen.tif')

In [9]:
raster.bounds

BoundingBox(left=-73.169644655, bottom=-46.394042378, right=-70.719691266, top=-43.705745621)

In [10]:
raster.crs

CRS.from_epsg(4326)

In [11]:
raster

<open DatasetReader name='/mnt/c/Users/R7 3700X/Documents/GitHub/VitoAlbornoz/Data/landcover_2018_aysen.tif' mode='r'>

In [12]:
mosaico = ee.Image("users/franciscoalbornoz/Mosaico_2018")
clasificacion_2018 = ee.Image("users/franciscoalbornoz/clasificacion_2018")

landsat_vis = {
    'min': 0,
    'max': 1,
    'bands': ['b5', 'b4', 'b3']
}
Map.addLayer(mosaico, landsat_vis, "Landsat-8-2018")
Map.addLayer(clasificacion_2018.randomVisualizer(), {}, "clasificación 2018")

Map

Map(bottom=21287.0, center=[-45.0498939995, -71.9446679605], controls=(WidgetControl(options=['position'], wid…

## Obteniendo los valores para el Training Datasets

In [ ]:
# points = mosaico.sample(**{
# #     'region': landsat.geometry(),
#     'scale': 30,
#     'numPixels': 20000,
#     'seed': 0,
#     'geometries': True  # Set this to False to ignore geometries
# })

# Map.addLayer(points, {}, 'training', False)
# Map

In [13]:
work_dir = os.path.expanduser('../Data/shapes/')
achaparrado_shp = os.path.join(work_dir, 'achaparrado_2018_2210_grande.shp')
agricolas_shp = os.path.join(work_dir, 'agricolas_2018_2610_grande.shp')
agua_shp = os.path.join(work_dir, 'agua_2018_0310_grande.shp')
arenalroca_shp = os.path.join(work_dir, 'arenal_roca_2018_0310_grande.shp')
bosquedegradado_shp = os.path.join(work_dir, 'bosques_degradados_2018_22112018.shp')
cuncuna_shp = os.path.join(work_dir, 'cuncunas_2018_2210_grande.shp')
estepa_shp = os.path.join(work_dir, 'estepa_2018_0310_grande.shp')
humedal_shp = os.path.join(work_dir, 'humedal_agua_2018_0310_grande.shp')
mallin_shp = os.path.join(work_dir, 'mallin_2018_0510_grande.shp')
matorralabierto_shp = os.path.join(work_dir, 'matorrales_abiertos_2018_22102018.shp')
matorraldenso_shp = os.path.join(work_dir, 'matorrales_densos_2018_22102018.shp')
plantacion1_shp = os.path.join(work_dir, 'plantaciones_2018_0310_grande.shp')
plantacion2_shp = os.path.join(work_dir, 'plantaciones_2018_23112018_grande.shp')
plantacion3_shp = os.path.join(work_dir, 'plantaciones_2018_26112018_grande.shp')
pradera_shp = os.path.join(work_dir, 'praderas_2018_0310_grande.shp')
primario_shp = os.path.join(work_dir, 'primario_2018_0310_grande.shp')
secundario_shp = os.path.join(work_dir, 'secundario_2018_0310_grande.shp')
sombra_shp = os.path.join(work_dir, 'sombras_topograficas_2018_0310_grande.shp')
suelo_shp = os.path.join(work_dir, 'suelo_desnudo_agricola_2018_0310_grande.shp')
merged_shp = os.path.join(work_dir, 'roi_merged.shp')

In [14]:
merged =geemap.shp_to_ee(merged_shp)
Map.addLayer(merged, {}, 'Áreas de entrenamiento')

In [15]:
dataset = mosaico.sampleRegions(**{
    'collection': merged,
    'properties': ['ID'],
    'scale' : 30 ,
})

In [16]:
print(dataset.size().getInfo())

14254


In [17]:
print(dataset.first().getInfo())

{'type': 'Feature', 'geometry': None, 'id': '0_0', 'properties': {'ID': 1, 'b1': 0.026168961077928543, 'b2': 0.04253531992435455, 'b3': 0.03470490872859955, 'b4': 0.33766427636146545, 'b5': 0.1495322287082672, 'b6': 0.06873124837875366, 'b7': 0.8135994672775269, 'b8': 0.7762474417686462, 'b9': 0.5209251642227173}}


### Dividiendo en muestras y datos de validación

In [19]:
# Guardamos las bandas
bands = ['b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8', 'b9']

# Esta propiedad guarda las clases de landcover
label = 'b1'

# Overlay the points on the imagery to get training.
# sample = landsat.select(bands).sampleRegions(**{
#   'collection': points,
#   'properties': [label],
#   'scale': 30
# })

# Adds a column of deterministic pseudorandom numbers. 
sample = dataset.randomColumn()

split = 0.7 

training = sample.filter(ee.Filter.lt('random', split))
validation = sample.filter(ee.Filter.gte('random', split))

In [20]:
training.first().getInfo()

{'type': 'Feature',
 'geometry': None,
 'id': '0_1',
 'properties': {'ID': 1,
  'b1': 0.02466532774269581,
  'b2': 0.03902462124824524,
  'b3': 0.03005797602236271,
  'b4': 0.31321027874946594,
  'b5': 0.13090473413467407,
  'b6': 0.061290204524993896,
  'b7': 0.8248717784881592,
  'b8': 0.7784170508384705,
  'b9': 0.5036694407463074,
  'random': 0.16340030157546792}}

In [21]:
validation.first().getInfo()

{'type': 'Feature',
 'geometry': None,
 'id': '0_0',
 'properties': {'ID': 1,
  'b1': 0.026168961077928543,
  'b2': 0.04253531992435455,
  'b3': 0.03470490872859955,
  'b4': 0.33766427636146545,
  'b5': 0.1495322287082672,
  'b6': 0.06873124837875366,
  'b7': 0.8135994672775269,
  'b8': 0.7762474417686462,
  'b9': 0.5209251642227173,
  'random': 0.9956806624347441}}

## Entrenando el modelo 

In [26]:

trained = ee.Classifier.smileCart().train(sample, label, bands)
classified = mosaico.select(bands).classify(trained)

In [27]:
trained

In [28]:
classified

### Clasificando la imagen

In [35]:
# Clasificando con las mismas bandas de entrenamiento
# result = landsat.select(bands).classify(classified)
class_palette = ['22c129',
 'f0ff00',
 '0c40f1',
 '685205',
 '0d4404',
 'e82d13',
 'f99504',
 '17d4c6',
 '1c6330',
 '24ff00',
 '2de50f',
 'ccba7d',
 'dec61a',
 '28d909',
 '03c2ff',
 '4e8c44',
 'fdfdfd',
 'c0c233'
]
# # Display the clusters with random colors.
Map.addLayer(classified,
             {'min': 0, 'max': 1, 'palette' : class_palette },
             'CART')
Map
             

EEException: Classifier training failed: 'Only one class.'.

In [32]:
class_values = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
    ]

In [33]:
land = classified.set('classification_class_values', class_values)
landcoverf = land.set('classification_class_palette', class_palette)

In [34]:
Map.addLayer(landcoverf, {}, 'Land cover SCART')
Map

EEException: Classifier training failed: 'Only one class.'.

### Categorizando según las clases 

In [ ]:
landcover.getInfo()

In [ ]:
class_values = [1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16]

In [ ]:
class_palette = ['476ba1',
 'd1defa',
 'decaca',
 'd99482',
 'ee0000',
 'ab0000',
 'b3aea3',
 '68ab63',
 '1c6330',
 'b5ca8f',
 'a68c30',
 'ccba7d',
 'e3e3c2',
 'caca78',
 '99c247',
 '78ae94']


In [ ]:
# landcoverf = result.set('classification_class_values', class_values)
# landcoverf = landcoverf.set('classification_class_palette', class_palette)

In [ ]:
# Map.addLayer(landcoverf, {}, 'Land cover clasificado' )
# Map

### Visualizando el resultado


In [ ]:
print('Change layer opacity:')
cluster_layer = Map.layers[-1]
cluster_layer.interact(opacity=(0, 1, 0.1))


In [ ]:
Map

### Agregando título al mapa

In [ ]:
# Map.add_legend('Clasificación supervisada CART')
# Map

### Evaluación de presición / Accuracy assessment

In [ ]:
# train_accuracy = classifier.confusionMatrix()

In [ ]:
# train_accuracy.getInfo()

In [ ]:
train_accuracy.accuracy().getInfo()

In [ ]:
train_accuracy.kappa().getInfo()

In [ ]:
train_accuracy.producersAccuracy().getInfo()

In [ ]:
train_accuracy.consumersAccuracy().getInfo()

#### Set de datos de validación

In [ ]:
validated = validation.classify(classifier)

In [ ]:
validated.first().getInfo()

In [ ]:
test_accuracy = validated.errorMatrix('b1', 'classification')

In [ ]:
test_accuracy.getInfo()

In [ ]:
test_accuracy.accuracy().getInfo()

In [ ]:
test_accuracy.kappa().getInfo()

In [ ]:
test_accuracy.producersAccuracy().getInfo()

In [ ]:
test_accuracy.consumersAccuracy().getInfo()

### Descargando la matrix

In [ ]:
import csv
import os

out_dir = os.path.join(os.path.expanduser('/mnt/c/Users/R7 3700X/'), 'Desktop')
training_csv = os.path.join(out_dir, 'train_accuracy.csv')
testing_csv = os.path.join(out_dir, 'test_accuracy.csv')

with open(training_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(train_accuracy.getInfo())
    
with open(testing_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(test_accuracy.getInfo())

### Exportando 

In [ ]:
import os
out_dir = os.path.join(os.path.expanduser('/mnt/c/Users/R7 3700X/'), 'Desktop')
out_file = os.path.join(out_dir, 'landcover.tif')

In [ ]:
geemap.ee_export_image(landcover, filename=out_file, scale=900)


In [ ]:
# geemap.ee_export_image_to_drive(landcover, description='landcover', folder='export', scale=900)